In [ ]:
""" Importing  Libraries """

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
""" 

NOTE : This is the original dataset

1. Header = None , which means There is no column name in the file
2. Engine = Python , This is to make sure Dataset gets imported correctly
3. Encoding = latin-1     , There are some special chracter in the movies title.
"""

movies = pd.read_csv("/content/drive/My Drive/Boltzmann Machine/ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv("/content/drive/My Drive/Boltzmann Machine/ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv("/content/drive/My Drive/Boltzmann Machine/ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
movies.head()

In [ ]:
"""
Here first column is the User_id
Second column is the Gender
Third column is the Age
and fourth column is , It seems some codes to the users job
"""

users.head()

In [ ]:
"""
First column is id
Second column is movies_id
third column is  Ratings (1 to 5)
fourth columns is Timestamp (Not really requirred)

"""

ratings.head()

# Data Preprocessing

In [ ]:
""" 
Preparing Training and Test Set 
NOte : Separater for this file is Tab hence using \t parameter
I am converting these dataset into numpy array because I am using PyTorch Tensors
"""

training_set = pd.read_csv("/content/drive/My Drive/Boltzmann Machine/ml-100k/u1.base", delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv("/content/drive/My Drive/U/Boltzmann Machine/ml-100k/u1.test", delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [ ]:
"""

Getting the total number of users and Movies, Why Am I doing this?

Because I will be converting my training_Set and test_Set into a matrix where lines are going to user and columns are 
going to be movies and cells are going to be the ratings.

I will create such matrix for training set and test set both

I will also include older users, older movies from original dataset into these two Matrixes 

And The Training set that we just imported, If a user did not rate the movies, We will put it a zero
into the cell of matrix that corresponds to  this user and that movie

"""

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [ ]:
"""

Going to convert our dataset into an array with users in lines and movies in columns

Why Am i doing this?

We will need to make a specific structure that will corresponds to  what the Restricted Boltzmann machine wants as input

These observations will go one by one into the RBM. Hence will create a structure that will contain these observations
that will go into the Network and their different features that are going to be into input nodes and that is what
i will do by creating this array with users in lines and movies in the columns because we will have observations in lines and 
features in the columns. 

In other words, It is the same like we do in all neural networks or ML ALgorithms.

I will accomplish this objective by creating a user defined function. I will give only one argument to this function
because it will apply to only one set of data which will training set first and when we will apply this to our training set 

or test set, We only need to replace this argument by training set and test set, Then colon :

after than I will tell the function what it needs to do


"""


def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
""" Converting the data into torch tensor """

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Preparing the Input for Boltzmann Machine

In [ ]:
""" Converting Ratings into Binary 1 liked and 0 not liked
Why am I doing this?
Because I will need to predict some Binary ratings 
But question is why do we also need input to have Binary input format?
Because RGM takes input vector and inside the input vector, It will predict the rating that was not originally rated by the user
But since these originally ratings are rated already by the user, Then the predicted rating output must have the same format.

Now I have used -1 here too, Let's understand why is that so?

Because Will replace all the zeroes in the original training set, ALl the ratings that were not actually existent These corresponds to the movies that
were not rated by the user but now since rating are gonna be zero and one, then Original zeroes must now have another value and this new value is gonna be 

minus one. So Minus one means that Movies were not rated

"""

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating Architecture of RBM

In [ ]:
""" Now I will build a RBM model, RBM is probabilistic Graphical Model, I will be using Class to build this model.

A CLass is model(Plan) of something we want to build, Let's say you want to build a house, YOU need model home, instructions and thats what a class will contain
 and the person who will make class will be the architect because He will define how a house should be built and once a class is made you can create

 some objects of this class. And object is nothing else but houses and that is exactly same for RBG model. I will be the architect for this model. I will make 

 class that will define how the RBM should be built Hence In this class. I will give all the information that I need to build the RBM, SO What are these

 information

 1. Number of hidden nodes
 2. weights for probability of the visible node given the hidden node
 3. bias for the same probability and bias for the probability of the visible node given the hidden node
 
 These are first parameters we need to initiaze the RBM. Hence Will put all this for first function called __init__ and then within the class we can add more things

 besides the parameters that defines the model. We can also add some functions.

 Just as an Architect can add more tools in making the model of the house. LIkewise I will write some function besides the __init__ function that will do all the 
 And these actions are to sample the Hidden nodes and visible nodes. 

 In Short, I will need around 3 functions

1. Function to initialize the RBM
2. Sample H that will sample the probabilities of the hidden nodes given the visible nodes
3. Function to sample v that will sample the probabilities of the visible nodes given the hidden nodes


SO one class and three functions to make 

Let's define the class using word 'class' and then class name, Make sure its name is in capital letter or at least starts with capital letter,
then parrenthesis and colon.
And As I have alredy talked above that first function we need to make is __init__ function hence will do that, This __init__ function has to be defined in
every class because This init function will have parameters of the object that will be  created once the class is made.
Here I Will have 3 arguments in __init__ function. First one will always be default parameters/argument that is self.

This self corresponds to the object that will be created afterwards and we need this argument because I will define some variables and we need to specify
These variables are the variables of the Object that will be created afterwards and not some global variables. 

And will put two another arguments. 1. NV - WHICH IS NUMBER OF VISIBLE Nodes and third is NH which is number of hidden nodes so these are just parameters of the __init__ function. 

And then I Will define what we need to initilize which is weights and bias hence now to define how this initialisation is going to work, I will need to add the column there :
there.

NOw within this __init__ function. We will initialize the parameters of our future objects that will create in this class. It will be all the parameters which
will be optimized during the training of the RBM.
""" 



class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  """
This last function is about constractive divergence  which is used to approximate the likelihood gradients, As we know that direct computation of the 

Gradient is too heavy therefore instead of computing it directly, I will try to approximate it. It comes with Gib Sampling consist of creating the Gibs chain

And this Gibs chain is created by sampling k times the hidden nodes and divisible nodes. And To make this, We will add four arguments

1. Input vector

2. vk (visible nodes optained after k sampling)

3. Ph0 that is vector of probabilities that is at the first

4. phk corresponds to  probabilities of the hidden nodes after k sampling

  """


  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)


"""
Now I will create the Object SO what do we do in Object is ...We just call the parameters nv and nh
"""

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)



# Training the RBM 

In [ ]:
""" Will create a loop which will go through each epochs batch passed through the network
 and at the end , Will get the final visible nodes will new ratings
 Will write the statement, Remember : Every Deep Learning algorithm need a loss function to measure the error, Then There is another for loop after Every batch
 of users going through the network. Rest everything is simple to understand hence not going to write everything here

"""
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))



# Testing the RBM

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

# Training RMSE

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

# Test RMSE

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        s += 1.
print('test loss: '+str(test_loss/s))